In [0]:
# !git clone https://github.com/KodeWil/final_project_AI_csv_Data.git

In [0]:
#Dependencies
# %reset
import keras
from keras.models import Sequential
from keras.layers import Dense# Neural network
import pandas as pd 
import glob 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import copy
from google.colab import files as fl
from sklearn.metrics import accuracy_score
from sklearn import neural_network
import random as rn


In [0]:
def addData(files: str) -> list:
    allCsv = []
    for file in files:
      temp = open(file)
      tempDf = pd.read_csv(file)
      allCsv.append(tempDf)
    return allCsv



In [0]:
def printAll(df):
  pd.set_option('display.max_rows', None)
  print(df)

In [249]:
frames = []

#Create a list ('frames') with four nested lists, one per sensor. 
#Each nested list have all the samples (DF) for that sensor.
%cd /content/final_project_AI_csv_Data/phone/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/phone/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
print(len(frames))
# 0 in frames = /phone/accel
# 1 in frames = /phone/gyro
# 2 in frames = /watch/accel
# 3 in frames = /watch/gyro

/content/final_project_AI_csv_Data/phone/accel
/content/final_project_AI_csv_Data/phone/gyro
/content/final_project_AI_csv_Data/watch/accel
/content/final_project_AI_csv_Data/watch/gyro
4


In [0]:
def cutSamples(frames1, frames2):
  for i in range (0,50):
    samples1 = frames1[i].shape[0]
    samples2 = frames2[i].shape[0]
    if(samples1 > samples2):
      toDrop = list(range(samples2, samples1))
      frames1[i].drop(frames1[i].index[[toDrop]], inplace = True)
    elif(samples2 > samples1):
      toDrop = list(range(samples1, samples2))
      frames2[i].drop(frames2[i].index[[toDrop]], inplace = True) 


In [0]:
def kerasSequiential(X, y):
  InpDim = X.shape[1]
  output = y.shape[1]
  model = Sequential()
  model.add(Dense(int(InpDim*2), input_dim=InpDim, activation="relu"))
  model.add(Dense(64, activation="relu"))
  model.add(Dense(output, activation="softmax"))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  history = model.fit(X, y, epochs=30, batch_size=40, verbose=0)
  return model

In [0]:
def mlpNN(Xt, yt, Xte, yte):
  # activTypes = ["logistic", "tanh", "relu"]
  # solverTypes = ["lbfgs", "sgd", "adam"]
  activTypes = ["relu"]
  solverTypes = ["adam"]
  allModels = []
  for activType in activTypes:
    for solverType in solverTypes:
      clf = neural_network.MLPClassifier(
          hidden_layer_sizes = [100,64,18],
          activation = activType,
          solver = solverType,
          alpha = 0.0001, # regularización, para la generalización del clasificador
          batch_size = 'auto',
          learning_rate_init = 0.001,
          power_t = 0.5,
          max_iter = 300,
          shuffle = True,
          verbose = False
          )
      allModels.append((clf, activType, solverType))
  best = 0
  bestIndex = -1
  for i, mlpModel in enumerate(allModels):
    mlpModel[0].fit(Xt, yt)
    y_predicho = mlpModel[0].predict(Xte)
    tempScore = accuracy_score(yte,y_predicho)
    if(tempScore > best):
      best = tempScore
      bestIndex = i
  activ = allModels[i][1]
  solve = allModels[i][2]
  print("")
  print("Scikit learn multilayer perceptron")
  print("")
  print("The best mlp's scikit learn model use activation as parameter " + activ + " as solver use " + solve + " and the accuracy score is : "+ str(best))
  print("")



In [0]:
#Split into train and test data, encode "ACTIVITY" column into a numerical column
def dataTransformation(frame, columns: bool):
  ohe = OneHotEncoder()
  sc = StandardScaler()
  if(columns):
    y = frame["ACTIVITY"]
    X = frame.drop(columns = ["ACTIVITY", "class"])
    X = X.values
    X = sc.fit_transform(X)
    y = y.values
    y = y.reshape(-1,1)
    y = ohe.fit_transform(y).toarray()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return (kerasSequiential(X_train, y_train), X_test, y_test)
  else:
    y = frame[0]
    y = y.values
    y = y.reshape(-1,1)
    y = ohe.fit_transform(y).toarray()
    X = frame.drop(columns = [0,91,93,182])
    X = X.values
    X = sc.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    mlpNN(X_train, y_train, X_test, y_test)
    return (kerasSequiential(X_train, y_train), X_test, y_test)






In [254]:
#Apply the models to some individuals samples 
scores = []
names = ["Phone Accel", "Phone Gyro" , "Watch Accel", "Watch Gyro"]
numSamples = 10
for frame in frames:
  tempAvgScore = 0
  for i in range(0,numSamples):
    tempVal = rn.randint(0,49)
    tempModel = dataTransformation(frame[tempVal], True)
    score, acc = tempModel[0].evaluate(tempModel[1], tempModel[2],
                            batch_size=20, verbose=0)
    tempAvgScore += acc
  
  scores.append(tempAvgScore/numSamples)
print("")
print("Using " + str(numSamples) + " samples (individual measurements) of each sensor " )
print("")
for i,name in enumerate(names):
  print("")
  print("The average accuracy score for " + name + " is: " + str(scores[i]))
  print("")  

# Concatenate all phone Dataframes in a single One
sensors = []
phoneAccel = pd.concat(frames[0])
sensors.append(phoneAccel)
phoneGyro = pd.concat(frames[1])
sensors.append(phoneGyro)
watchAccel = pd.concat(frames[2])
sensors.append(watchAccel)
watchGyro = pd.concat(frames[3])
sensors.append(watchGyro)


#Cut samples to match with the number of samples in the other sensor (accel and gyro)
cutSamples(frames[0], frames[1]) #Phone sensors
cutSamples(frames[2], frames[3]) #Watch sensors

#Concat all individual samples for a single sensor in one data frame. 
phoneAccel = pd.concat(frames[0], ignore_index=True)
phoneGyro = pd.concat(frames[1], ignore_index=True)
watchAccel = pd.concat(frames[2], ignore_index=True)
watchGyro = pd.concat(frames[3], ignore_index=True)

#Create a new data frame with both sensors for both devices.
phone = pd.concat([phoneAccel, phoneGyro], ignore_index=True, axis = 1)
phoneIndex = phone[phone[0] != phone[93]].index
phone.drop(phoneIndex, inplace=True)
watch = pd.concat([watchAccel, watchGyro], ignore_index=True, axis = 1)
watchIndex = watch[watch[0] != watch[93]].index
watch.drop(watchIndex, inplace=True)

models = []
# for sensor in sensors:
#   models.append(dataTransformation(sensor, True))
print("")
print("Phone merged sensors accuracy")
print("")
models.append(dataTransformation(phone, False))
print("")
print("Watch merged sensors accuracy")
print("")
models.append(dataTransformation(watch, False))

# names = ["Phone Accel", "Phone Gyro" , "Watch Accel", "Watch Gyro", "Phone", "Watch" ]
# print("")
print("Models' scores")
print("")

for i, model in  enumerate(models):
  score, acc = model[0].evaluate(model[1], model[2],
                            batch_size=20, verbose=0)
  print("")
  print("The accuracy for " + names[i]+ " is: " + str(acc))
  print("")



Using 10 samples (individual measurements) of each sensor 


The average accuracy score for Phone Accel is: 0.9291931927204132


The average accuracy score for Phone Gyro is: 0.7389419198036193


The average accuracy score for Watch Accel is: 0.8990736722946167


The average accuracy score for Watch Gyro is: 0.8066042840480805



/usr/local/lib/python3.6/dist-packages/pandas/core/indexes/base.py:3940: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)



Phone merged sensors accuracy


Scikit learn multilayer perceptron

The best mlp's scikit learn model use activation as parameter relu as solver use adam and the accuracy score is : 0.7642752562225475


Watch merged sensors accuracy


Scikit learn multilayer perceptron

The best mlp's scikit learn model use activation as parameter relu as solver use adam and the accuracy score is : 0.7976151043391851

Models' scores


The accuracy for Phone Accel is: 0.8696925044059753


The accuracy for Phone Gyro is: 0.8668433427810669

